## Stream features using Signals

This notebook creates a new feature view using the SDK that will be computed using stream processing.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Stream processing/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

In [ ]:
from snowplow_signals import Signals

sp_signals = Signals(api_url='https://0fcfdf97-6447-4208-8cd0-39f82befbd07.svc.snplow.net')

### Define a new feature

This block creates a single feature definition including the logic how it should be calculated (it's filters and aggregation).

The feature calculates the number of add to cart ecommerce events.

In [ ]:
from snowplow_signals import (
    Feature,
    FilterCombinator,
    FilterCondition,
)

products_added_to_cart_feature = Feature(
    name="products_added_to_cart",
    dtype="STRING_LIST",
    events=[
        "iglu:com.snowplowanalytics.snowplow.ecommerce/snowplow_ecommerce_action/jsonschema/1-0-2"
    ],
    type="unique_list",
    property="contexts_com_snowplowanalytics_snowplow_ecommerce_product_1[0].name",
    scope="session",
    filter=FilterCombinator(
        combinator="and",
        condition=[
            FilterCondition(
                property="unstruct_event_com_snowplowanalytics_snowplow_ecommerce_snowplow_ecommerce_action_1:type",
                operator="equals",
                value="add_to_cart",
            ),
        ],
    ),
)

### Wrapping the feature in a feature view

All features need to be included in feature views that can be considered as "tables" of features.

Feature views are immutable and versioned.

In [ ]:
from snowplow_signals import FeatureView, session_entity

feature_view = FeatureView(
    name="my_ecommerce_features",
    version=1,
    entities=[
        session_entity,
    ],
    features=[
        products_added_to_cart_feature,
    ],
)

### Testing the feature view

Execute the feature view on the last one hour of data from the atomic events table to verify that it works correctly.

In [ ]:
data = sp_signals.test(
    feature_view=feature_view,
    app_ids=["website"],
)
data

### Applying the feature view to Signals

The following block pushes the feature view definition to the Signals API and makes it available for processing.

In [ ]:
sp_signals.apply([feature_view])

### Track Snowplow events to test the feature view

Track Snowplow events to test that the feature view is producing data.

In [ ]:
from snowplow_tracker import Snowplow

tracker = Snowplow.create_tracker(
    namespace="ns3",
    endpoint="https://aws-sandbox-dev1.collector.snplow.net",
    app_id="ai_demo",
)

In [ ]:
from snowplow_tracker import SelfDescribing, SelfDescribingJson, Subject

subject = Subject()
subject.set_domain_user_id("2718de92-3a9d-4ad7-a8da-76a7b0f5c3db")
subject.set_domain_session_id("30ec7e22-04cd-4dca-9162-b26c3a6001c1")

event = SelfDescribing(
    SelfDescribingJson(
        "iglu:com.snowplowanalytics.snowplow.ecommerce/snowplow_ecommerce_action/jsonschema/1-0-2",
        {"type": "add_to_cart"},
    ),
    subject
)

event.context.append(
    SelfDescribingJson(
        "iglu:com.snowplowanalytics.snowplow.ecommerce/product/jsonschema/1-0-0",
        {
            "id": "12345",
            "name": "shoe",
            "category": "shoes",
            "price": 100.0,
            "quantity": 1,
            "currency": "USD",
        },
    )
)

event.context.append(
    SelfDescribingJson(
        "iglu:com.snowplowanalytics.snowplow.ecommerce/cart/jsonschema/1-0-0",
        {
            "total_value": 212,
            "currency": "USD",
        },
    )
)

tracker.track(event)
tracker.flush()

### Retrieving data

One can fetch the latest feature values for a particular session from the feature view as follow.

In [ ]:
response = sp_signals.get_online_features(
    features=[feature_view],
    entity="30ec7e22-04cd-4dca-9162-b26c3a6001c1",
    entity_type_id="session",
)

response.to_dataframe()